# Genetic Partitioning

This notebook shows the implementation of the algorithm proposed in [Generalised Circuit Partitioning for Distributed Quantum Computing](https://arxiv.org/abs/2408.01424). 

In [9]:
from qiskit.circuit.library import QFT, QuantumVolume
from disqco.circuits.cp_fraction import cp_fraction, cz_fraction
from disqco.circuits.QAOA import QAOA_random
from qiskit import transpile
from QASMBench.interface.qiskit import QASMBenchmark
from disqco.graphs.quantum_network import QuantumNetwork


num_qubits = 16

circuit = cp_fraction(num_qubits, num_qubits, fraction=0.5)

# circuit = QAOA_random(num_qubits, prob=0.5, reps=1)

# circuit = QFT(num_qubits, do_swaps=False)

# circuit = QuantumVolume(num_qubits, 10)

num_partitions = 4 # Define the number of partitions or QPUs you will distribute over
depth = circuit.depth()

qpu_size = num_qubits // num_partitions + 1 # Define the number of qubits per QPU. For simplicity, we divide qubits evenly and add one additional space to each for teleportation (an extra space is needed for the teleportation qubit so the algorithmr requires it!)
qpu_sizes = [qpu_size] * num_partitions # Store the number of qubits per QPU in a list

# Create a quantum network with the specified number of qubits and partitions

# If we do not specificy the connectivity, we have all-to-all by default.

quantum_network = QuantumNetwork(qpu_sizes)


basis_gates = ['u', 'cp']

# Transpile the circuit to the basis gates
circuit = transpile(circuit, basis_gates=basis_gates)

print(f'Number of qubits in circuit {circuit.num_qubits}')
print(f'Circuit depth: {circuit.depth()}')

Number of qubits in circuit 16
Circuit depth: 16


The algorithm below uses the same parameters used in the papers for GCP-E. The GCP-S version, which doesn't use gate packing / gate grouping can be accessed by setting gate_packing = False. The multi_process flag can be used to parallelise the genetic algorithm which will speed up large instances.

In [10]:
from disqco.parti.genetic.genetic_algorithm_original import Genetic_Partitioning

genetic_partitioner = Genetic_Partitioning(circuit, qpu_sizes, gate_packing=True)


results = genetic_partitioner.run(pop_size=100,num_generations=100,mutation_rate=0.9, multi_process=True, log_frequency=10, search_method=True,search_number=100)



Current best cut: 37
Current best cut: 33
Current best cut: 33
Current best cut: 33
Current best cut: 33
Current best cut: 33
Current best cut: 33
Current best cut: 33
Current best cut: 33
Current best cut: 33


In [11]:
from disqco.circuit_extraction.circuit_extractor import PartitionedCircuitExtractor
from disqco.graphs.QC_hypergraph import QuantumCircuitHyperGraph

graph = QuantumCircuitHyperGraph(circuit)

extractor = PartitionedCircuitExtractor(graph=graph, 
                                        network=quantum_network,
                                        partition_assignment=results['best_assignment'])

partitioned_circuit = extractor.extract_partitioned_circuit()

partitioned_circuit.draw(output='mpl', style='bw', fold=100)
